In [71]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [72]:
ls

MLNinthPractical.ipynb  wheat-seeds.csv


In [73]:
cd /gdrive

/gdrive


In [74]:
ls

'My Drive'/


In [75]:
cd 'My Drive'/'Colab Notebooks'/'ml_p9'/

/gdrive/My Drive/Colab Notebooks/ml_p9


In [76]:
ls

MLNinthPractical.ipynb  wheat-seeds.csv


In [0]:
import pandas as pd
import numpy as np

In [78]:
df = pd.read_csv('wheat-seeds.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [79]:
column_names = ['area', 'perimeter', 'compactness', 'length of kernel', 'width of kernel', 'asymmetry coefficient', 'length of kernel groove','kernel type']
column_names

['area',
 'perimeter',
 'compactness',
 'length of kernel',
 'width of kernel',
 'asymmetry coefficient',
 'length of kernel groove',
 'kernel type']

In [80]:
df.columns = column_names
df.head()

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,kernel type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [81]:
print(df.isnull().sum())

area                       0
perimeter                  0
compactness                0
length of kernel           0
width of kernel            0
asymmetry coefficient      0
length of kernel groove    0
kernel type                0
dtype: int64


In [82]:
df.describe()

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,kernel type
count,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,14.847524,14.559286,0.870999,5.628533,3.258605,3.700201,5.408071,2.000000
std,2.909699,1.305959,0.023629,0.443063,0.377714,1.503557,0.491480,0.818448
min,10.590000,12.410000,0.808100,4.899000,2.630000,0.765100,4.519000,1.000000
25%,12.270000,13.450000,0.856900,5.262250,2.944000,2.561500,5.045000,1.000000
50%,14.355000,14.320000,0.873450,5.523500,3.237000,3.599000,5.223000,2.000000
75%,17.305000,15.715000,0.887775,5.979750,3.561750,4.768750,5.877000,3.000000
max,21.180000,17.250000,0.918300,6.675000,4.033000,8.456000,6.550000,3.000000


In [83]:
for col in df.columns:
  print(df[col].dtype)

float64
float64
float64
float64
float64
float64
float64
int64


In [84]:
df.corr()

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,kernel type
area,1.000000,0.994341,0.608288,0.949985,0.970771,-0.229572,0.863693,-0.346058
perimeter,0.994341,1.000000,0.529244,0.972422,0.944829,-0.217340,0.890784,-0.327900
compactness,0.608288,0.529244,1.000000,0.367915,0.761635,-0.331471,0.226825,-0.531007
length of kernel,0.949985,0.972422,0.367915,1.000000,0.860415,-0.171562,0.932806,-0.257269
width of kernel,0.970771,0.944829,0.761635,0.860415,1.000000,-0.258037,0.749131,-0.423463
asymmetry coefficient,-0.229572,-0.217340,-0.331471,-0.171562,-0.258037,1.000000,-0.011079,0.577273
length of kernel groove,0.863693,0.890784,0.226825,0.932806,0.749131,-0.011079,1.000000,0.024301
kernel type,-0.346058,-0.327900,-0.531007,-0.257269,-0.423463,0.577273,0.024301,1.000000


In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [86]:
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.1.0


In [87]:
raw_dataset = pd.read_csv('wheat-seeds.csv', names=column_names,
                      na_values = " ", comment='\t',
                      sep=",", skipinitialspace=True)

dataset = raw_dataset.copy()

dataset.tail() 

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,kernel type
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3
209,12.30,13.34,0.8684,5.243,2.974,5.637,5.063,3


In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [0]:
train_labels = train_dataset.pop('kernel type')
test_labels = test_dataset.pop('kernel type')

In [0]:
train_labels = train_labels - 1
test_labels = test_labels - 1

## So that the labels are 0, 1, 2 instead of 1, 2, 3

In [91]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
area,168.0,14.850060,2.850291,10.5900,12.29000,14.44500,17.140000,20.9700
perimeter,168.0,14.560952,1.275165,12.4100,13.46500,14.38000,15.662500,17.2500
compactness,168.0,0.871299,0.024270,0.8081,0.85730,0.87395,0.887925,0.9183
length of kernel,168.0,5.625387,0.433835,4.8990,5.26275,5.53400,5.984500,6.6750
width of kernel,168.0,3.259893,0.374640,2.6480,2.95100,3.25150,3.531750,4.0320
asymmetry coefficient,168.0,3.641721,1.527259,0.7651,2.46175,3.52950,4.685500,8.4560
length of kernel groove,168.0,5.399649,0.491331,4.5190,5.04400,5.22050,5.877000,6.5500


In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [0]:
model = keras.Sequential([
    layers.Dense(96, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')
  ])

cce = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(loss=cce, optimizer=tf.keras.optimizers.RMSprop(0.001), metrics=['accuracy'])

In [97]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 96)                768       
_________________________________________________________________
dense_68 (Dense)             (None, 64)                6208      
_________________________________________________________________
dense_69 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_70 (Dense)             (None, 3)                 99        
Total params: 9,155
Trainable params: 9,155
Non-trainable params: 0
_________________________________________________________________


In [98]:
from keras.callbacks import EarlyStopping

EPOCHS = 100

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=1,
  callbacks=[es])

Train on 134 samples, validate on 34 samples
Epoch 1/100
134/134 [==============================] - 1s 4ms/sample - loss: 0.9871 - accuracy: 0.7537 - val_loss: 0.8098 - val_accuracy: 1.0000
Epoch 2/100
134/134 [==============================] - 0s 180us/sample - loss: 0.7533 - accuracy: 0.8881 - val_loss: 0.6145 - val_accuracy: 0.9706
Epoch 3/100
134/134 [==============================] - 0s 167us/sample - loss: 0.5938 - accuracy: 0.9104 - val_loss: 0.4867 - val_accuracy: 0.9706
Epoch 4/100
134/134 [==============================] - 0s 169us/sample - loss: 0.4798 - accuracy: 0.9030 - val_loss: 0.3854 - val_accuracy: 0.9706
Epoch 5/100
134/134 [==============================] - 0s 164us/sample - loss: 0.3923 - accuracy: 0.9179 - val_loss: 0.3164 - val_accuracy: 0.9706
Epoch 6/100
134/134 [==============================] - 0s 196us/sample - loss: 0.3331 - accuracy: 0.9104 - val_loss: 0.2538 - val_accuracy: 0.9706
Epoch 7/100
134/134 [==============================] - 0s 177us/sample - lo

In [99]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
23,0.136410,0.940298,0.082244,0.970588,23
24,0.132057,0.940298,0.098332,0.970588,24
25,0.128120,0.932836,0.086282,0.970588,25
26,0.121650,0.947761,0.087818,0.970588,26
27,0.116668,0.947761,0.091610,0.970588,27


In [104]:
test_predictions = model.predict(normed_test_data)

test_predictions


array([[7.68089116e-01, 2.29986981e-01, 1.92380429e-03],
       [9.43435848e-01, 1.47141330e-02, 4.18500900e-02],
       [9.90570307e-01, 8.79629329e-03, 6.33376651e-04],
       [8.19464862e-01, 6.40853168e-03, 1.74126610e-01],
       [8.36103261e-01, 3.85691412e-02, 1.25327632e-01],
       [4.36562181e-01, 5.56191504e-01, 7.24632200e-03],
       [6.78516805e-01, 1.06284292e-02, 3.10854703e-01],
       [9.45239127e-01, 3.99415344e-02, 1.48192793e-02],
       [9.76627231e-01, 1.89603474e-02, 4.41241870e-03],
       [9.81082916e-01, 4.37439559e-03, 1.45426365e-02],
       [4.08481713e-03, 9.95859087e-01, 5.61896195e-05],
       [1.17300423e-02, 9.88026738e-01, 2.43232731e-04],
       [6.06781323e-05, 9.99939322e-01, 4.48635973e-08],
       [6.64828345e-02, 9.32883322e-01, 6.33910007e-04],
       [1.45169511e-03, 9.98543739e-01, 4.66261417e-06],
       [1.65300080e-04, 9.99834418e-01, 2.08640373e-07],
       [3.32606811e-04, 9.99666452e-01, 9.13954580e-07],
       [4.84607153e-05, 9.99951

In [105]:
for i in range(len(test_predictions)):
  print(test_predictions[i].argmax())

0
0
0
0
0
1
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
0
2
2
2
2
2
2
2
2
2
2
2
2
0
2
